#RNN

##import/load

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
#text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
#print('Length of text: {} characters'.format(len(text)))

In [3]:
with open('/content/drive/My Drive/sherlock.txt', 'r') as f: 
    text = f.read()

print('Length of text: {} characters'.format(len(text)))

Length of text: 3381928 characters


In [4]:
print(text[:250])





                          THE COMPLETE SHERLOCK HOLMES

                               Arthur Conan Doyle



                                Table of contents

               A Study In Scarlet

               The Sign of the Four

              


##vectorize

In [5]:
# The unique characters
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

97 unique characters


In [6]:
# map unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '&' :   4,
  "'" :   5,
  '(' :   6,
  ')' :   7,
  '*' :   8,
  ',' :   9,
  '-' :  10,
  '.' :  11,
  '/' :  12,
  '0' :  13,
  '1' :  14,
  '2' :  15,
  '3' :  16,
  '4' :  17,
  '5' :  18,
  '6' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\n\n\n\n         ' ---- characters mapped to int ---- > [0 0 0 0 1 1 1 1 1 1 1 1 1]


In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'\n\n\n\n                          THE COMPLETE SHERLOCK HOLMES\n\n                               Arthur Con'
'an Doyle\n\n\n\n                                Table of contents\n\n               A Study In Scarlet\n\n   '
'            The Sign of the Four\n\n                  The Adventures of Sherlock Holmes\n               '
'A Scandal in Bohemia\n               The Red-Headed League\n               A Case of Identity\n         '
'      The Boscombe Valley Mystery\n               The Five Orange Pips\n               The Man with the'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n\n\n\n                          THE COMPLETE SHERLOCK HOLMES\n\n                               Arthur Co'
Target data: '\n\n\n                          THE COMPLETE SHERLOCK HOLMES\n\n                               Arthur Con'


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('\n')
  expected output: 0 ('\n')
Step    1
  input: 0 ('\n')
  expected output: 0 ('\n')
Step    2
  input: 0 ('\n')
  expected output: 0 ('\n')
Step    3
  input: 0 ('\n')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 1 (' ')


##base model

In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           24832     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ruded out of the floor. Then it was withdrawn as\n     suddenly as it appeared, and all was dark agai'

Next Char Predictions: 
 '65ñKP\'o;;kOüP.96mFEs:àRi’3!/`LñâHLN7ôgkq["lB6733:9N8zo]x£7.3M]vzTnxCCKK j`S?4dt½nCQîq`UûfyülséTYß,à&'


In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5767837


In [23]:
model.compile(optimizer='adam', loss=loss)

## epoch

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
EPOCHS = 10

In [26]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
523/523 [==============================] - 33s 58ms/step - loss: 2.6100
Epoch 2/10
523/523 [==============================] - 32s 60ms/step - loss: 1.4129
Epoch 3/10
523/523 [==============================] - 33s 60ms/step - loss: 1.2427
Epoch 4/10
523/523 [==============================] - 32s 60ms/step - loss: 1.1735
Epoch 5/10
523/523 [==============================] - 32s 60ms/step - loss: 1.1310
Epoch 6/10
523/523 [==============================] - 33s 60ms/step - loss: 1.0966
Epoch 7/10
523/523 [==============================] - 33s 60ms/step - loss: 1.0707
Epoch 8/10
523/523 [==============================] - 32s 60ms/step - loss: 1.0492
Epoch 9/10
523/523 [==============================] - 33s 60ms/step - loss: 1.0277
Epoch 10/10
523/523 [==============================] - 32s 60ms/step - loss: 1.0096


In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            24832     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 97)             99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


## generate

In [30]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [31]:
print(generate_text(model, start_string=u"Sherlock "))

Sherlock headed is
     descended, and with a sailor country building and face he feads
     an emergen land, they would seize up a juncof it once more to ask, when I had
     readily in it, impenefally used as his body ladiments considerably wealthy the
     gentleman. I human with this point would be to extreord. They
     talk efore old dream--such as turning two
     eyes, practical spocial towards us--"

     Holmes toicess with this hiding-place.

     "They have glared the advice. His dressing-room
     gave no scowling glancing that he could accept a little for the
     nt was but him."

     "Then we bead like the essentials of what you don't know that there
     was down An electround if you can send your brain to CurKh

     The Croweral Square. Amus add the words of the edge of something fect. His face
     still was unlikely to him there. Holmes give
     you all in which my mele rattle how you would be more. They
     struck home leaving with a real murder at the world, a

In [32]:
print(generate_text(model, start_string=u"When"))

When we
     have fred on accounts to her rugge and second
     projecting designs the galloys of finding the daily
     damp-backed door, and heavily beyond one of this evil murder.

     "I can see d." He did somehow all three seemed across the retirns that I
     had let the uge of traceid Inly deceives the agent
     disappointed. The clergyman, but within our man, with a
     cabable in his rooms."

     The bbother had become expecially, once or no--orgeon the moor.
     Here we have decayed so we now, and Godfrey was to be despair. I shall guide
     three inclusive residence in the friends beside by the short and
     sallowly dropping rapidly. A shadow was close to the burotest
     face
     himself which I have not
     learned to show you have help out in my direction without the suspicion, the towns
     which led to the lid. And me under the lady. "Ah, yes--the fatal effort. We caught one
     of the event for me to the lawy, and in reading the hat
     you wore a migst o

In [33]:
print(generate_text(model, start_string=u"discover"))

discovering with
     admiration, and had once beced the teral to
     leave, then! that surly it all, played these, were removed young man, with
     McMurdo and a badx on earth to get round to it will be back by reward to chord. Mycroft, it is true, too! I thought your motive cannot be more. After all.
     But the front, silver, taking that Willable sheer best had rung-ink.

     "It is losely that we may not go so hard when they were leaving to us. There is
     any in their smiling as he replaced it therence. May I ask.
     Sherlock Hallshires had been kissed in has a connection with
     the wind-wood of the room, and mack before we would be taken, and
     made frame of him, Miss Stapleton had books immissive
     o'clock. We are greatly at the time later mad over to it. You know me. What's this, it's
     question has not disappeared."

     "What was it?" asked Holmes.

     "Why, you are jointed out some trade!"

     "Yes, it was a door that I'll shafe that her solution com

## functionize 

In [34]:
def iteration(embed_dim,rnn_units,epochs):
  dataset = sequences.map(split_input_target)
  dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

  vocab_size = len(vocab)
  embedding_dim = embed_dim
  rnn_units = rnn_units

  model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

  model.summary()

  model.compile(optimizer='adam', loss=loss)

  checkpoint_dir = './training_checkpoints'
  checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_prefix,
      save_weights_only=True)

  history = model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])

  tf.train.latest_checkpoint(checkpoint_dir)

  print("\n---\n")

  model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

  model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)).expect_partial()

  model.build(tf.TensorShape([1, None]))

  model.summary()

  print(generate_text(model, start_string=u"Sherlock"))

  print("\n---\n")

  print(generate_text(model, start_string=u"When"))

  print("\n---\n")

  print(generate_text(model, start_string=u"discover"))

#RNN Variations

## 256 512 10

In [35]:
iteration(256,512,10)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           24832     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 512)           1182720   
_________________________________________________________________
dense_2 (Dense)              (64, None, 97)            49761     
Total params: 1,257,313
Trainable params: 1,257,313
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
523/523 [==============================] - 17s 28ms/step - loss: 2.6967
Epoch 2/10
523/523 [==============================] - 16s 28ms/step - loss: 1.5412
Epoch 3/10
523/523 [==============================] - 16s 28ms/step - loss: 1.3411
Epoch 4/10
523/523 [==============================] - 16s 28ms/step - loss: 1.2577
Epoch 5/10
523/523 [======================

##256 256 10

In [36]:
iteration(256,256,10)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           24832     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 256)           394752    
_________________________________________________________________
dense_4 (Dense)              (64, None, 97)            24929     
Total params: 444,513
Trainable params: 444,513
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
523/523 [==============================] - 12s 18ms/step - loss: 2.5659
Epoch 2/10
523/523 [==============================] - 10s 18ms/step - loss: 1.5750
Epoch 3/10
523/523 [==============================] - 10s 18ms/step - loss: 1.4061
Epoch 4/10
523/523 [==============================] - 10s 18ms/step - loss: 1.3330
Epoch 5/10
523/523 [==========================

##256 128 10

In [37]:
iteration(256,128,10)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           24832     
_________________________________________________________________
gru_6 (GRU)                  (64, None, 128)           148224    
_________________________________________________________________
dense_6 (Dense)              (64, None, 97)            12513     
Total params: 185,569
Trainable params: 185,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
523/523 [==============================] - 10s 16ms/step - loss: 2.6971
Epoch 2/10
523/523 [==============================] - 9s 15ms/step - loss: 1.6935
Epoch 3/10
523/523 [==============================] - 9s 15ms/step - loss: 1.5386
Epoch 4/10
523/523 [==============================] - 9s 15ms/step - loss: 1.4651
Epoch 5/10
523/523 [=============================

##256 64 10

In [38]:
iteration(256,64,10)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (64, None, 256)           24832     
_________________________________________________________________
gru_8 (GRU)                  (64, None, 64)            61824     
_________________________________________________________________
dense_8 (Dense)              (64, None, 97)            6305      
Total params: 92,961
Trainable params: 92,961
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
523/523 [==============================] - 10s 15ms/step - loss: 2.8922
Epoch 2/10
523/523 [==============================] - 8s 14ms/step - loss: 1.8443
Epoch 3/10
523/523 [==============================] - 8s 14ms/step - loss: 1.6884
Epoch 4/10
523/523 [==============================] - 8s 14ms/step - loss: 1.6202
Epoch 5/10
523/523 [==============================]

#Embed Variations

##64 1024 10

In [39]:
iteration(64,1024,10)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (64, None, 64)            6208      
_________________________________________________________________
gru_10 (GRU)                 (64, None, 1024)          3348480   
_________________________________________________________________
dense_10 (Dense)             (64, None, 97)            99425     
Total params: 3,454,113
Trainable params: 3,454,113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
523/523 [==============================] - 28s 50ms/step - loss: 2.7950
Epoch 2/10
523/523 [==============================] - 27s 50ms/step - loss: 1.5539
Epoch 3/10
523/523 [==============================] - 28s 52ms/step - loss: 1.3100
Epoch 4/10
523/523 [==============================] - 28s 52ms/step - loss: 1.2155
Epoch 5/10
523/523 [=====================

##128 1024 10

In [40]:
iteration(128,1024,10)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (64, None, 128)           12416     
_________________________________________________________________
gru_12 (GRU)                 (64, None, 1024)          3545088   
_________________________________________________________________
dense_12 (Dense)             (64, None, 97)            99425     
Total params: 3,656,929
Trainable params: 3,656,929
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
523/523 [==============================] - 32s 57ms/step - loss: 2.7672
Epoch 2/10
523/523 [==============================] - 31s 58ms/step - loss: 1.4820
Epoch 3/10
523/523 [==============================] - 31s 58ms/step - loss: 1.2745
Epoch 4/10
523/523 [==============================] - 31s 58ms/step - loss: 1.1922
Epoch 5/10
523/523 [=====================

##512 1024 10

In [41]:
iteration(512,1024,10)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (64, None, 512)           49664     
_________________________________________________________________
gru_14 (GRU)                 (64, None, 1024)          4724736   
_________________________________________________________________
dense_14 (Dense)             (64, None, 97)            99425     
Total params: 4,873,825
Trainable params: 4,873,825
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
523/523 [==============================] - 39s 70ms/step - loss: 2.4977
Epoch 2/10
523/523 [==============================] - 38s 70ms/step - loss: 1.4292
Epoch 3/10
523/523 [==============================] - 38s 70ms/step - loss: 1.2568
Epoch 4/10
523/523 [==============================] - 38s 70ms/step - loss: 1.1882
Epoch 5/10
523/523 [=====================

##1024 1024 10

In [42]:
iteration(1024,1024,10)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (64, None, 1024)          99328     
_________________________________________________________________
gru_16 (GRU)                 (64, None, 1024)          6297600   
_________________________________________________________________
dense_16 (Dense)             (64, None, 97)            99425     
Total params: 6,496,353
Trainable params: 6,496,353
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
523/523 [==============================] - 49s 89ms/step - loss: 2.1713
Epoch 2/10
523/523 [==============================] - 48s 90ms/step - loss: 1.2877
Epoch 3/10
523/523 [==============================] - 48s 90ms/step - loss: 1.1819
Epoch 4/10
523/523 [==============================] - 48s 90ms/step - loss: 1.1298
Epoch 5/10
523/523 [=====================

#Epoch Variations

##256 1024 5

In [43]:
iteration(256,1024,5)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (64, None, 256)           24832     
_________________________________________________________________
gru_18 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_18 (Dense)             (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
523/523 [==============================] - 34s 60ms/step - loss: 2.6004
Epoch 2/5
523/523 [==============================] - 32s 60ms/step - loss: 1.4282
Epoch 3/5
523/523 [==============================] - 33s 60ms/step - loss: 1.2517
Epoch 4/5
523/523 [==============================] - 33s 60ms/step - loss: 1.1792
Epoch 5/5
523/523 [==========================

##256 1024 20

In [44]:
iteration(256,1024,20)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (64, None, 256)           24832     
_________________________________________________________________
gru_20 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_20 (Dense)             (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
523/523 [==============================] - 34s 60ms/step - loss: 2.6359
Epoch 2/20
523/523 [==============================] - 32s 60ms/step - loss: 1.4205
Epoch 3/20
523/523 [==============================] - 33s 60ms/step - loss: 1.2441
Epoch 4/20
523/523 [==============================] - 33s 60ms/step - loss: 1.1754
Epoch 5/20
523/523 [=====================

##256 1024 30

In [45]:
iteration(256,1024,30)

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (64, None, 256)           24832     
_________________________________________________________________
gru_22 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_22 (Dense)             (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
523/523 [==============================] - 34s 60ms/step - loss: 2.6191
Epoch 2/30
523/523 [==============================] - 32s 60ms/step - loss: 1.4280
Epoch 3/30
523/523 [==============================] - 32s 60ms/step - loss: 1.2491
Epoch 4/30
523/523 [==============================] - 32s 60ms/step - loss: 1.1771
Epoch 5/30
523/523 [=====================

##256 1024 40

In [46]:
iteration(256,1024,40)

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (64, None, 256)           24832     
_________________________________________________________________
gru_24 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_24 (Dense)             (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
523/523 [==============================] - 33s 60ms/step - loss: 2.6119
Epoch 2/40
523/523 [==============================] - 32s 60ms/step - loss: 1.4319
Epoch 3/40
523/523 [==============================] - 32s 60ms/step - loss: 1.2524
Epoch 4/40
523/523 [==============================] - 33s 60ms/step - loss: 1.1816
Epoch 5/40
523/523 [=====================

##256 1024 50

In [47]:
iteration(256,1024,50)

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (64, None, 256)           24832     
_________________________________________________________________
gru_26 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_26 (Dense)             (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
523/523 [==============================] - 33s 60ms/step - loss: 2.6322
Epoch 2/50
523/523 [==============================] - 32s 60ms/step - loss: 1.4288
Epoch 3/50
523/523 [==============================] - 32s 60ms/step - loss: 1.2487
Epoch 4/50
523/523 [==============================] - 32s 60ms/step - loss: 1.1777
Epoch 5/50
523/523 [=====================

## MAX

In [48]:
iteration(1024,1024,40)

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (64, None, 1024)          99328     
_________________________________________________________________
gru_28 (GRU)                 (64, None, 1024)          6297600   
_________________________________________________________________
dense_28 (Dense)             (64, None, 97)            99425     
Total params: 6,496,353
Trainable params: 6,496,353
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
523/523 [==============================] - 49s 89ms/step - loss: 2.0714
Epoch 2/40
523/523 [==============================] - 47s 89ms/step - loss: 1.2657
Epoch 3/40
523/523 [==============================] - 48s 89ms/step - loss: 1.1708
Epoch 4/40
523/523 [==============================] - 47s 89ms/step - loss: 1.1244
Epoch 5/40
523/523 [=====================